<a href="https://colab.research.google.com/github/etan15/PrivateLLM-Enchanted/blob/main/Run_Ollama_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FIRST Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh

In [3]:
#SECOND Get Ngrok authentication token from colab secrets environment
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
#THIRD aiohttp for concurrent subprocess execution
!pip install aiohttp pyngrok

In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', NGROK_AUTH_TOKEN]),
)

In [ ]:
# Run multiple tasks concurrently:
#  1. Start the Ollama server.
#  2. Start ngrok to forward HTTP traffic from the local ollama api running on localhost:11434.
#     Instructions come from Ollama doc: https://github.com/ollama/ollama/blob/main/docs/faq.md#how-can-i-use-ollama-with-ngrok
await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'pull', 'dolphin-llama3']),
    run_process(['ollama', 'pull', 'hermes3']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434', '--domain', 'organic-driving-drum.ngrok-free.app']),
)

*
